First import all necessairy modules. 

In [12]:
from preprocess import Preprocessor
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.metrics import Accuracy, Precision, Recall

The we load the data, preprocess it and then split it into Training and Test data. This may take a while. 

In [13]:
pp = Preprocessor([i for i in range(745, 760)], logs_per_class=100, window_size=20, volatile=False)
train_data, test_data = pp.stratified_split()
X_train, y_train = zip(*train_data)
X_test, y_test = zip(*test_data)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

All states have the desired log count


Now we define the LSTM model, train it and then print a summary.

In [27]:
# Build the model
lstm_layers = 5
epochs = 1000
model = Sequential()
model.add(LSTM(lstm_layers, input_shape=(pp.window_size, 5)))  # 50 units in LSTM layer
num_classes = len(set(y_train))  # Assuming labels are integers
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
              metrics=[
                  'accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=32)

# Summary of the model
model.summary()

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1491 - loss: 1.3947
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1527 - loss: 1.3551 
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2831 - loss: 1.3876 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2959 - loss: 1.3571 
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2900 - loss: 1.3423 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3763 - loss: 1.3407 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5180 - loss: 1.3347
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5086 - loss: 1.3386 
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4756 - loss: 1.3398 
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5070 - loss: 1.3200 
Epoch 11/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4595 - loss: 1.3200 
Epoch 12/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_11 (LSTM)                  │ (None, 5)              │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │            24 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 734 (2.87 KB)

 Trainable params: 244 (976.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 490 (1.92 KB)

In [24]:
model.save(f"lstm_x{lstm_layers}_{epochs}.h5")

Then we test the model with the test data

In [25]:
test_loss = model.evaluate(X_test, y_test)
print(f"Test MSE: {test_loss}")

print()
results = model.evaluate(X_test, y_test, verbose=0)

for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")



predictions = model.predict(X_test)

# Print a few predictions vs actuals
a = []
for i in range(80):
    a.append(np.argmax(predictions[i]) == y_test[i])
    # print(f"Predicted: {predictions[i]}, Actual: {y_test[i]}")

print(a.count(True))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6016 - loss: 0.8065 
Test MSE: [0.7879094481468201, 0.5625]

loss: 0.7879
compile_metrics: 0.5625


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
45
